**The following needs to be done for submission**

1. Either the model downloaded in training must be uploaded(I have uploaded in weight folder u can create new one to test yours) or notebook dataset can be refered from Add data --> under Notebook Output    Files and accordingly path of model needs to be changed (tf.keras.models.load_model('../input/weight/model_Resnet50.h5', custom_objects = None )).
2. Switch on the Accelerator --> GPU
3. Internet must be disabled
4. Try Run All 
5. Onc successfully completed click on Save Version in top right corner to confirm 3 things -  whether under advanced setting (1) always save output is checked along with (2) Run it with GPU and (3) Save and Run All options is selected. Once it is confirmed click on Save
6. Once the process is completed now you can submit your work either from submit predictions in cassave competition main page or click on the version number (after Save Version) and go to viewer and under 3 dots submit your work. Within few minutes you must get score >= 68%

In [ ]:
import tensorflow as tf
import re
import numpy as np

In [ ]:
DIM = 128
BATCH_SIZE = 64

In [ ]:
TEST_FILENAMES = tf.io.gfile.glob('../input/cassava-leaf-disease-classification/test_tfrecords/*.tfrec') 
print(TEST_FILENAMES)


In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  
    image = tf.image.resize(image, [DIM, DIM])
    image = tf.reshape(image, [DIM, DIM, 3])
    return image

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
       "image": tf.io.FixedLenFeature([], tf.string), 
       "image_name": tf.io.FixedLenFeature([], tf.string), 
       
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['image_name']
    return image, idnum 

def load_dataset(filenames, labeled = True, ordered = False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 
        
    dataset = tf.data.TFRecordDataset(filenames) 
    dataset = dataset.with_options(ignore_order) 
    dataset = dataset.map(read_unlabeled_tfrecord) 
    return dataset

def get_test_dataset(ordered=True):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TEST_IMAGES = int( count_data_items(TEST_FILENAMES) )

def predict():
    model = tf.keras.models.load_model('../input/weight/model_Resnet50.h5', custom_objects = None ) 
    print(model.summary())
    test_ds = get_test_dataset(ordered=True) 
    test_images_ds = test_ds.map(lambda image, idnum: image)
    prob = model.predict(test_images_ds)
    predictions = np.argmax(prob, axis=-1)
    print(predictions)
    test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
    test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') 
    np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='image_id,label', comments='')
predict()
print("Done")